<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/01-Mi-segundo-modelo-Plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Mi segundo modelo
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Nombre y Apellido &bull; Fecha
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

Este cuaderno está diseñado practicar en el entrenamiento y evaluación de un modelo de regresión lineal utilizando datos reales. Los objetivos principales son:

- Entrenar un modelo de regresión lineal.
- Evaluar el ajuste del modelo.


Del conjunto de datos disponible [aquí](https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv), se desea predecir la variable `price` en función de las variables `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`.

In [15]:
# Paquetes necesarios
import numpy as np  # Para generación de datos y operaciones numéricas
import pandas as pd  # Para manejo de datos estructurados
import matplotlib.pyplot as plt  # Para visualización de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression  # Para el modelo de regresión lineal
from sklearn.metrics import mean_squared_error, r2_score  # Para evaluación del modelo
from sklearn.model_selection import train_test_split

---
## <font color='264CC7'> Lectura de Datos </font>


Realiza la lectura de los datos y presenta las primeras filas del conjunto de datos.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv')
#Elegimos solo las variables de estudio:
df_modelo = df[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']]
df_modelo.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,221900.0,3,1.00,1180,5650,1.0
1,538000.0,3,2.25,2570,7242,2.0
2,180000.0,2,1.00,770,10000,1.0
3,604000.0,4,3.00,1960,5000,1.0
4,510000.0,3,2.00,1680,8080,1.0


---
## <font color='264CC7'> Preprocesamiento de datos </font>

Realiza un escalado de los datos, prueba diferentes técnicas y selecciona la que mejor se ajuste a los datos.

In [4]:
#Dividimos los datos entre caracteristicas y etiqueta
X = df_modelo[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']]
y = df_modelo['price']
#Realizamos un sondeo de las variables para ver como podemos proceder con el escalado:
print(X.describe())

           bedrooms     bathrooms   sqft_living      sqft_lot        floors
count  21613.000000  21613.000000  21613.000000  2.161300e+04  21613.000000
mean       3.370842      2.114757   2079.899736  1.510697e+04      1.494309
std        0.930062      0.770163    918.440897  4.142051e+04      0.539989
min        0.000000      0.000000    290.000000  5.200000e+02      1.000000
25%        3.000000      1.750000   1427.000000  5.040000e+03      1.000000
50%        3.000000      2.250000   1910.000000  7.618000e+03      1.500000
75%        4.000000      2.500000   2550.000000  1.068800e+04      2.000000
max       33.000000      8.000000  13540.000000  1.651359e+06      3.500000


In [16]:
#De acuerdo a esto, vemos que sqft_living y sqft_lot tienen una desviación estándar muy grande a comparación de los otros datos.
#Dada esta dispersión se ve importante reducirla y la manera más efectiva es mediante una estandarización
#(reducimos escala pero evitamos sobrerepresentacion de datos)
st_scaler = StandardScaler()
X_scaled = pd.DataFrame(st_scaler.fit_transform(X), columns=X.columns)
#ahora vemos los valores:
print("Normalizacion por estandarización")
display(X_scaled.head())
#Para comparabilidad realizamos los otros dos métodos (por xmax y por xmax xmin)
# Normalización por el máximo
x_max = X.max()
X_max_normalized = X / x_max
print("Normalización por el máximo:\n")
display(X_max_normalized.head())

# Normalización Min-Max
min_max_scaler = MinMaxScaler()
X_min_max_normalized = pd.DataFrame(min_max_scaler.fit_transform(X), columns=X.columns)
print("Normalización Min-Max:\n")
display(X_min_max_normalized.head())

Normalizacion por estandarización


,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,-0.398737,-1.447464,-0.979835,-0.228321,-0.915427
1,-0.398737,0.175607,0.533634,-0.189885,0.936506
2,-1.473959,-1.447464,-1.426254,-0.123298,-0.915427
3,0.676485,1.149449,-0.130550,-0.244014,-0.915427
4,-0.398737,-0.149007,-0.435422,-0.169653,-0.915427


Normalización por el máximo:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.087149,0.003421,0.285714
1,0.090909,0.28125,0.189808,0.004385,0.571429
2,0.060606,0.12500,0.056869,0.006056,0.285714
3,0.121212,0.37500,0.144756,0.003028,0.285714
4,0.090909,0.25000,0.124077,0.004893,0.285714


Normalización Min-Max:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.067170,0.003108,0.0
1,0.090909,0.28125,0.172075,0.004072,0.4
2,0.060606,0.12500,0.036226,0.005743,0.0
3,0.121212,0.37500,0.126038,0.002714,0.0
4,0.090909,0.25000,0.104906,0.004579,0.0


---
## <font color='264CC7'> Características y etiquetas </font>

Separa las características y las etiquetas y mostrar la dimensión de cada elemento.

---
## <font color='264CC7'> Conjunto de entrenamiento y prueba </font>

Divide los datos en dos conjuntos: uno para entrenamiento y otro para prueba y visualiza la dimensión de cada conjunto. Usar `random_state` igual al últmo número de tu cédula.

Revisa la distribución de las etiquetas en los conjuntos de entrenamiento y prueba.

---
## <font color='264CC7'> Entrenamiento </font>

Define un modelo de regresión lineal y entrenalo con los datos de entrenamiento.

---
## <font color='264CC7'> Evaluación </font>

Realiza la predicciones con el conjunto de prueba y muestra los valores en una tabla.

Calcula el error cuadrático medio.

Escribe el error cuadrático medio para cada tipo de escalado:

- Escalado MaxAbsScaler:
- Escalado MinMaxScaler:
- Escalado RobustScaler:
- Escalado StandardScaler:
- Escalado Normalizer:
- Escalado QuantileTransformer:
- Escalado PowerTransformer:

Escribe un párrafo sobre qué hace cada escalado no visto en clase:

- **Escalado ...:** ...